In [1]:
import tensorflow as tf

In [2]:
daily_sales_number = [23, 22, -108, 31, -1, 32, 34, 31]

In [3]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_number)

In [4]:
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [5]:
for sales in tf_dataset :
    print(sales)

tf.Tensor(23, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-108, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)
tf.Tensor(-1, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(34, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)


In [6]:
for sales in tf_dataset :
    print(sales.numpy())

23
22
-108
31
-1
32
34
31


In [7]:
def print_sales(tf_dataset) : 
    for sales in tf_dataset.as_numpy_iterator() :
        print(sales)

In [8]:
for sales in tf_dataset.take(3) : 
    print(sales.numpy())

23
22
-108


In [9]:
tf_dataset = tf_dataset.filter(lambda x : x>0)

In [10]:
print_sales(tf_dataset)

23
22
31
32
34
31


In [11]:
tf_dataset = tf_dataset.map(lambda x : x*77)
print_sales(tf_dataset)

1771
1694
2387
2464
2618
2387


In [12]:
tf_dataset = tf_dataset.shuffle(2)
print_sales(tf_dataset)

1694
1771
2464
2387
2618
2387


In [13]:
for sales_batch in tf_dataset.batch(2) : 
    print(sales_batch.numpy())

[1771 2387]
[1694 2618]
[2387 2464]


In [14]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_number)

tf_dataset =  tf_dataset.filter(lambda x : x >0). map(lambda y : y*77).shuffle(2).batch(2)

print_sales(tf_dataset)

[1694 2387]
[1771 2464]
[2618 2387]


In [15]:
# for image in images_ds.take(3) : 
#     print(image)

In [16]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle = False)

def print_images(images_ds) : 
    for image in images_ds : 
        print(image.numpy())
        
print_images(images_ds.take(3))

b'images\\cheetah_images\\1_000_ Cheetah Pictures.jpg'
b'images\\cheetah_images\\1_366 Cheetah Running Stock Photos.jpg'
b'images\\cheetah_images\\212_255 Cheetah Images_ Stock Photos (1).jpg'


In [17]:
images_ds = images_ds.shuffle(200)

In [18]:
print_images(images_ds.take(3))

b'images\\lion_images\\A Day in the Life of a Lion Tracker.jpg'
b'images\\lion_images\\world lion day_ World Lion Day 2022.jpg'
b'images\\cheetah_images\\Cheetah_ Faster than most cars but low.jpeg'


In [19]:
class_names = ['cat', 'dog']

In [20]:
image_count = len(images_ds)
image_count

186

In [21]:
train_size = int(image_count*0.8)

train_ds = images_ds.take(train_size)

test_ds = images_ds.skip(train_size)

In [22]:
len(train_ds)

148

In [23]:
len(test_ds)

38

In [24]:
img = 'images\\cheetah_images\\Welcome Cheetahs to India _ MyGov.in.jpg'

In [25]:
img.split('\\')[1]

'cheetah_images'

In [26]:
# Or
img.split('\\')[-2]

'cheetah_images'

In [27]:
import os
def get_label(file_path) : 
    return tf.strings.split(file_path, os.path.sep)[-2] 

In [28]:
# for t in train_ds.take(4) :
#     print(t.numpy())

In [29]:
# for label in train_ds.map(get_label) : 
#     print(label)

In [30]:
def process_image(file_path) :
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    
    return img, label

In [31]:
train_ds = train_ds.map(process_image)
for img, label in train_ds.take(3) : 
    print('Image :', img)
    print('label :', label)

Image : tf.Tensor(
[[[ 85.17969   127.828125   79.00745  ]
  [ 28.889526   49.537964   39.565308 ]
  [ 46.407288   63.407288   53.407288 ]
  ...
  [ 49.759888   60.9953     47.967957 ]
  [ 66.26642   113.425476   75.84595  ]
  [  5.9804688  61.188538   14.815674 ]]

 [[ 31.012817  108.         40.00641  ]
  [  1.6403198  53.64032     7.6240845]
  [ 12.746094   63.691406   30.800781 ]
  ...
  [ 38.65155    43.65155    36.65155  ]
  [ 59.217834  101.530945   60.64032  ]
  [ 23.186707   81.324524   30.393433 ]]

 [[  0.         54.839844    0.       ]
  [ 30.733582   83.73358    31.733582 ]
  [ 11.520813   72.759094   13.829407 ]
  ...
  [ 51.7865     73.591125   50.65625  ]
  [ 25.539062   70.53906    29.539062 ]
  [ 44.338013   60.213013   42.275513 ]]

 ...

 [[  1.1151123  90.0672      3.211731 ]
  [  5.4000244  92.56641    25.266418 ]
  [ 81.39844   105.36456    66.19269  ]
  ...
  [110.37891   111.37891   106.37891  ]
  [126.        127.        122.       ]
  [116.81561   122.81561 

In [32]:
def scale(image, label) : 
    return image/255, label

In [33]:
train_ds = train_ds.map(scale)
for img, label in train_ds.take(3) : 
    print('Image :', img.numpy()[0][0])
    print('label :', label.numpy())

Image : [0.26286766 0.25502452 0.17267157]
label : b'cheetah_images'
Image : [0.         0.01471366 0.02647837]
label : b'cheetah_images'
Image : [0.02074274 0.01682117 0.00113489]
label : b'lion_images'
